In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [2]:
train_data = pd.read_csv("./smile_description_train.csv")
test_data = pd.read_csv("./smile_description_test.csv")

In [3]:
# Replacing Null values of train and test data with 0 
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [4]:
train_data

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,label,Assay_id
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1644
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2,2451
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,2,1384
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75378,11.460021,-3.868472,11.460021,0.053611,0.712426,230.245,220.165,230.036128,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,33
75379,5.928972,-2.841623,5.928972,0.082346,0.720533,313.747,296.611,313.041677,104.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1632
75380,4.975926,0.848333,4.975926,0.848333,0.596343,167.258,162.218,166.986341,50.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1373
75381,10.241948,0.324028,10.241948,0.324028,0.519485,128.215,112.087,128.120115,54.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [5]:
# Split into X and y
X = train_data.drop("label",axis=1)
y = train_data["label"]
X = pd.get_dummies(X,columns=["Assay_id"])
test_data = pd.get_dummies(test_data,columns=["Assay_id"])

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [7]:
def create_submission(predict,filename):
    sub_file = pd.read_csv("./data/sample_submission.csv")
    sub_file["Predicted"] = predict
    sub_file.to_csv(filename,index=False)
    print(filename," Created")

In [9]:
def f1_score_xg_boost(X,y,learning_rate=None):
    skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    cv =  skf.get_n_splits(X, y)
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
    cv_results = cross_validate(xgb.XGBClassifier(random_state=42,n_estimators=700,eta=learning_rate), X, y, cv=cv,scoring=scoring,verbose=3,n_jobs=-1)
    print("F1 score with ",sum(cv_results["test_f1_score"])/5)
    print("Accuracy score with ",sum(cv_results["test_accuracy"])/5)

In [10]:
import numpy as np
def get_count_of_ones_and_twos(predict):
    print("Number of predicted ones",np.count_nonzero(predict==1))
    print("Number of predicted twos",np.count_nonzero(predict==2))

In [18]:
f1_score_xg_boost(X,y,0.35)
"""
n_estimators = 700
learning_rate = 0.35
F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841
n_estimators = 900
learning_rate = 0.4
F1 score with  0.9472493912457081
Accuracy score with  0.9082816785922123
"""

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 30.4min remaining: 45.6min


F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 30.4min finished


'\nn_estimators = 700\nlearning_rate = 0.35\nF1 score with  0.9483140858093458\nAccuracy score with  0.9100460248164841\nn_estimators = 900\nlearning_rate = 0.4\nF1 score with  0.9472493912457081\nAccuracy score with  0.9082816785922123\n'

In [16]:
xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=700,eta=0.35)

In [17]:
xgb_c.fit(X,y)
predict = xgb_c.predict(test_data)
predict_real = label_encoder.inverse_transform(predict)

In [18]:
get_count_of_ones_and_twos(predict_real)

Number of predicted ones 1186
Number of predicted twos 9808


In [19]:
create_submission(predict_real,"submission_1_mar_24.csv")

submission_1_mar_24.csv  Created
